In [1]:
from lib.datasets import ATPBind3D

# dataset_orig = ATPBind3D()
# dataset_orig.initialize_mask_and_weights()

In [7]:
dataset = ATPBind3D(to_slice=True)
dataset.initialize_mask_and_weights()

Initialize Undersampling: all ones
Initialize Weighting: all ones


ATPBind3D(
  #sample: 631
  #task: 1
)

In [35]:
from lib.pipeline import Pipeline
import pandas as pd
from torchdrug import utils, data
import os
import torch
import inspect
import numpy as np

from lib.utils import generate_mean_ensemble_metrics_auto, read_initial_csv, aggregate_pred_dataframe
GPU = 0

from atpbind_main import main


In [2]:
main(model_key='esm-t33', valid_fold=0)

batch_size: 8
init pipeline, model: esm-t33, dataset: atpbind3d, gpus: [0]
load model esm-t33, kwargs: {'gpu': 0, 'freeze_esm': False, 'freeze_layer_count': 30}
get dataset atpbind3d
Initialize Undersampling: all ones
Initialize Weighting: all ones
train samples: 302, valid samples: 76, test samples: 41
use cyclic lr scheduler
pipeline batch_size: 8
0m20s {'sensitivity': 0.5949, 'precision': 0.5902, 'mcc': 0.5702, 'micro_auprc': 0.6137, 'valid_mcc': 0.541, 'train_bce': 0.166, 'valid_bce': 0.1067, 'lr': 0.00084}
0m19s {'sensitivity': 0.6938, 'precision': 0.6601, 'mcc': 0.6586, 'micro_auprc': 0.7047, 'valid_mcc': 0.6246, 'train_bce': 0.0828, 'valid_bce': 0.0937, 'lr': 0.00138}
0m20s {'sensitivity': 0.6204, 'precision': 0.6946, 'mcc': 0.6389, 'micro_auprc': 0.6339, 'valid_mcc': 0.6264, 'train_bce': 0.0429, 'valid_bce': 0.1536, 'lr': 0.00192}
0m21s {'sensitivity': 0.6715, 'precision': 0.6651, 'mcc': 0.6501, 'micro_auprc': 0.6658, 'valid_mcc': 0.6298, 'train_bce': 0.0323, 'valid_bce': 0.124

In [1]:
from lib.pipeline import Pipeline
from torchdrug import utils, data
import torch
import numpy as np

CYCLE_SIZE = 10
pipeline = Pipeline(
    dataset='atpbind3d',
    model='lm-gearnet',
    gpus=[1],
    model_kwargs={
        'gpu': 1,
        'lm_type': 'esm-t33',
        'gearnet_hidden_dim_size': 512,
        'gearnet_hidden_dim_count': 4,
        'lm_freeze_layer_count': 30,
    },
    scheduler='cyclic',
    scheduler_kwargs={
        'base_lr': 3e-4,
        'max_lr': 3e-3,
        'step_size_up': CYCLE_SIZE / 2,
        'step_size_down': CYCLE_SIZE / 2,
        'cycle_momentum': False
    },
    batch_size=8,
    dataset_kwargs={
        'to_slice': True,
        'max_slice_length': 650,
        'padding': 50,
    },
    valid_fold_num=0,
)

init pipeline, model: lm-gearnet, dataset: atpbind3d, gpus: [1]
load model lm-gearnet, kwargs: {'gpu': 1, 'lm_type': 'esm-t33', 'gearnet_hidden_dim_size': 512, 'gearnet_hidden_dim_count': 4, 'lm_freeze_layer_count': 30}
freeze_lm: 30
get dataset with max_length: 650, kwargs: {'to_slice': True, 'max_slice_length': 650, 'padding': 50}
get dataset atpbind3d
Initialize Undersampling: all ones
Initialize Weighting: all ones
train samples: 332, valid samples: 84, test samples: 41
use cyclic lr scheduler
pipeline batch_size: 8


In [2]:
pipeline.train_until_fit(patience=10, max_epoch=10)

evaluate test sliced with threshold: -0.3999999999999999, max_slice_length: 650, padding: 50
1m1s {'sensitivity': 0.436, 'precision': 0.6599, 'mcc': 0.5188, 'micro_auprc': 0.5559, 'valid_mcc': 0.512, 'train_bce': 0.1818, 'valid_bce': 0.1203, 'lr': 0.00084}
evaluate test sliced with threshold: -1.0999999999999999, max_slice_length: 650, padding: 50
0m59s {'sensitivity': 0.6295, 'precision': 0.5127, 'mcc': 0.5451, 'micro_auprc': 0.5876, 'valid_mcc': 0.6111, 'train_bce': 0.0756, 'valid_bce': 0.0855, 'lr': 0.00138}
evaluate test sliced with threshold: -0.09999999999999964, max_slice_length: 650, padding: 50
0m58s {'sensitivity': 0.6473, 'precision': 0.5746, 'mcc': 0.5899, 'micro_auprc': 0.6375, 'valid_mcc': 0.6133, 'train_bce': 0.0495, 'valid_bce': 0.0984, 'lr': 0.00192}
evaluate test sliced with threshold: -1.0, max_slice_length: 650, padding: 50
0m59s {'sensitivity': 0.5729, 'precision': 0.6975, 'mcc': 0.6163, 'micro_auprc': 0.6295, 'valid_mcc': 0.6264, 'train_bce': 0.0303, 'valid_bce': 

[{'sensitivity': 0.436,
  'precision': 0.6599,
  'mcc': 0.5188,
  'micro_auprc': 0.5559,
  'valid_mcc': 0.512,
  'train_bce': 0.1818,
  'valid_bce': 0.1203,
  'lr': 0.00084},
 {'sensitivity': 0.6295,
  'precision': 0.5127,
  'mcc': 0.5451,
  'micro_auprc': 0.5876,
  'valid_mcc': 0.6111,
  'train_bce': 0.0756,
  'valid_bce': 0.0855,
  'lr': 0.00138},
 {'sensitivity': 0.6473,
  'precision': 0.5746,
  'mcc': 0.5899,
  'micro_auprc': 0.6375,
  'valid_mcc': 0.6133,
  'train_bce': 0.0495,
  'valid_bce': 0.0984,
  'lr': 0.00192},
 {'sensitivity': 0.5729,
  'precision': 0.6975,
  'mcc': 0.6163,
  'micro_auprc': 0.6295,
  'valid_mcc': 0.6264,
  'train_bce': 0.0303,
  'valid_bce': 0.1206,
  'lr': 0.00246},
 {'sensitivity': 0.6131,
  'precision': 0.6398,
  'mcc': 0.6087,
  'micro_auprc': 0.6371,
  'valid_mcc': 0.6487,
  'train_bce': 0.0203,
  'valid_bce': 0.1023,
  'lr': 0.003},
 {'sensitivity': 0.5759,
  'precision': 0.6973,
  'mcc': 0.6178,
  'micro_auprc': 0.6213,
  'valid_mcc': 0.6257,
  'tra

In [3]:
from lib.datasets import get_slices, protein_to_slices
from lib.utils import dict_tensor_to_num, round_dict

dataset = pipeline.dataset

from torchdrug import transforms
pipeline.dataset.transform = transforms.ProteinView(view='residue')

train_set, valid_set, test_set = pipeline.dataset.split()

def infer_sliced(pipeline, protein, max_slice_length=650, padding=50):
    GPU = 1
    target = protein.target
    intermediate_preds = []
    sliced_proteins, _ = protein_to_slices(protein, target, max_slice_length=max_slice_length, padding=padding)
    dataloader = data.DataLoader(sliced_proteins, batch_size=1, shuffle=False)
    with torch.no_grad():
        for batch in dataloader:
            batch = utils.cuda(batch, device=torch.device(f'cuda:{GPU}'))
            pred = pipeline.task.predict({"graph": batch})
            intermediate_preds.append(pred)
    final_preds = torch.zeros(target.shape)
    for i, (start, end) in enumerate(get_slices(target.shape[0], max_slice_length=max_slice_length, padding=padding)):
        final_preds[start:end] += intermediate_preds[i].cpu()
        if i > 0:
            final_preds[start:start+padding] /= 2
    return final_preds



def get_pred_and_target_with_sliced_dataset(pipeline, data_set):
    preds = []
    targets = []
    for item in data_set:
        pred = infer_sliced(pipeline, item['graph'])
        preds.append(pred)

        dataloader = data.DataLoader([item], batch_size=1, shuffle=False)    
        target = pipeline.task.target(next(iter(dataloader)))
        targets.append(target)
        
    pred = utils.cat(preds)
    target = utils.cat(targets)
    return pred, target


pred, target = get_pred_and_target_with_sliced_dataset(pipeline, test_set)

print(pred)

metric = pipeline.task.evaluate(pred, target, threshold=-2.9)
round_dict(dict_tensor_to_num(metric), 4)

tensor([[ -8.8587],
        [ -7.7427],
        [ -9.4689],
        ...,
        [-14.3080],
        [-13.4456],
        [-13.2479]])


{'sensitivity': 0.635,
 'precision': 0.6926,
 'mcc': 0.6479,
 'micro_auprc': 0.6661}

In [4]:
metric = pipeline.task.evaluate(pred, target, threshold=-3)
round_dict(dict_tensor_to_num(metric), 4)

{'sensitivity': 0.635,
 'precision': 0.6903,
 'mcc': 0.6467,
 'micro_auprc': 0.6661}

In [8]:
pipeline.valid_dataset_stats()

{'best_mcc': 0.6806598102694253,
 'best_threshold': -1.0999999999999999,
 'loss': 0.16789284483952957}

In [6]:
def get_best_mcc(pipeline, valid_set):
    pred, target = get_pred_and_target_with_sliced_dataset(pipeline, valid_set)
    thresholds = np.linspace(-3, 1, num=41)
    mcc_values = [pipeline.task.evaluate(pred, target, threshold=th)['mcc'] for th in thresholds]
    
    max_mcc_idx = np.argmax(mcc_values)
    print(mcc_values)
    return round(thresholds[max_mcc_idx], 2), mcc_values[max_mcc_idx]


best_threshold, best_mcc = get_best_mcc(pipeline, valid_set)
print(best_threshold, best_mcc)

[0.6719496756019859, 0.6737841390670234, 0.6739642661708714, 0.6745887111112963, 0.6753388171835185, 0.6746966688503381, 0.6750844733485658, 0.6757992916735809, 0.6759438984696856, 0.6761720045849945, 0.6752558695999953, 0.6749297830832559, 0.6760232566910069, 0.6761153466905574, 0.6769829930959705, 0.6772850143018433, 0.6773653668131454, 0.6780504060088259, 0.6795738626874842, 0.6806598102694253, 0.6788067006387218, 0.6788478563459069, 0.6783976359865879, 0.6790427861905656, 0.6787444424793512, 0.6778632103042203, 0.6772055924489395, 0.6784831411937146, 0.6764918169585021, 0.6768154760072012, 0.6771480966802296, 0.6755352387470382, 0.6754955418239035, 0.6748646138546883, 0.6734310253912156, 0.6710088450901409, 0.6702001563890424, 0.6685808301139903, 0.667958744321504, 0.6669304098434377, 0.6681347185682568]
-1.1 0.6806598102694253


In [9]:
metric = pipeline.task.evaluate(pred, target, threshold=best_threshold)
round_dict(dict_tensor_to_num(metric), 4)

{'sensitivity': 0.5964,
 'precision': 0.7585,
 'mcc': 0.6591,
 'micro_auprc': 0.6661}

In [10]:
pipeline.evaluate_test_sliced(best_threshold)

evaluate test sliced with threshold: -1.1, max_slice_length: 650, padding: 50


{'sensitivity': 0.5964391827583313,
 'precision': 0.7584905624389648,
 'mcc': 0.6590645086899897,
 'micro_auprc': 0.6661270260810852}

In [20]:
import pandas as pd

df = pd.read_csv('result_cv/result_cv.csv')

In [22]:
df.groupby(['max_slice_length', 'padding']).agg({'mcc': ['mean', 'std', 'count']}).sort_values(('mcc', 'mean'), ascending=False)

mcc                
                              mean       std count
max_slice_length padding                          
350              75       0.681467  0.013155     9
                 125      0.679011  0.014514     9
450              50       0.674170  0.011377    10
350              100      0.673311  0.013633     9
                 50       0.673022  0.012201     9
450              75       0.671733  0.011259     9
                 100      0.671556  0.016372     9
650              75       0.669920  0.015827    10
550              50       0.668040  0.013633    10
650              50       0.667750  0.008605    10
550              100      0.666610  0.016278    10
                 125      0.665340  0.010795    10
                 75       0.665220  0.014115    10
650              100      0.662700  0.014697    10
450              125      0.661733  0.013594     9
650              125      0.657500  0.012605    10

In [21]:
df.query('model_key == "esm-33-gearnet"').groupby(['model_key', 'max_slice_length', 'padding']).agg({'mcc': ['mean', 'std', 'count']}).sort_values(('mcc', 'mean'), ascending=False)

mcc                
                                             mean       std count
model_key      max_slice_length padding                          
esm-33-gearnet 350              75       0.687250  0.011487     4
                                125      0.685950  0.013252     4
               450              100      0.681800  0.013404     4
                                75       0.677050  0.013139     4
                                50       0.676560  0.008816     5
               350              50       0.676250  0.015425     4
               650              75       0.674980  0.016582     5
               550              100      0.674520  0.015245     5
               350              100      0.673475  0.016287     4
               550              50       0.671980  0.011995     5
                                75       0.671020  0.016264     5
                                125      0.670980  0.008949     5
               650              50       0.670540  0.010050     5
               450              125      0.667675  0.017608     4
               650              100      0.665900  0.016261     5
                                125      0.665560  0.006495     5

In [15]:
df.query('model_key == "esm-t33"').groupby(['model_key', 'max_slice_length', 'padding']).agg({'mcc': ['mean', 'std', 'count']}).sort_values(('mcc', 'mean'), ascending=False)

mcc                
                                       mean       std count
model_key max_slice_length padding                         
esm-t33   350              75       0.67684  0.013672     5
                           125      0.67346  0.014246     5
                           100      0.67318  0.013143     5
          450              50       0.67178  0.014116     5
          350              50       0.67044  0.010027     5
          450              75       0.66748  0.008555     5
          650              50       0.66496  0.006794     5
                           75       0.66486  0.014989     5
          550              50       0.66410  0.015345     5
          450              100      0.66336  0.014575     5
          550              125      0.65970  0.010128     5
          650              100      0.65950  0.014000     5
          550              75       0.65942  0.009982     5
                           100      0.65870  0.014401     5
          450              125      0.65698  0.008575     5
          650              125      0.64944  0.012365     5